In [ ]:
# IMPORTANT: The parameters below are set only for running this notebook independently. 
# When executing the full Ploomber pipeline, these values will be overridden by the settings in `pipeline.yaml`. 
# Any modifications made here will not persist when running the pipeline.
upstream = None
COUNTRY =  'ARM' # Code of the Country
product = {'data': f'../data/processed/{COUNTRY}/cp_funds_and_sdg.xlsx'}  # Path to save the final data product (stored under the 'data' key)
data_source = ['data/raw/insight-programme-strategic-sdg-expenditure/insight-programme-strategic-sdg-activity-sp-2018-2021.xlsx',
            'data/raw/insight-programme-strategic-sdg-expenditure/insight-programme-strategic-sdg-activity-sp-2022-2025.xlsx',
            'data/raw/insight-programme-strategic-sdg-expenditure/insight-programme-strategic-sdg-activity-sp-2016-2017.xlsx']


This Notebook maps Expenses by CP, Source and SDG (Sustainable Develoment Goals) using activities.

In [ ]:
import pandas as pd
import numpy as np
import re
from unicef_cpe.config import PROJ_ROOT
from unicef_cpe.utils import get_ecaro_countries_mapping
from unicef_cpe.plotting import SDG_goals

In [ ]:
country_map = {k:v for k,v in get_ecaro_countries_mapping(priority=False).items() if k in COUNTRY}
country_code_map = {v:k for k,v in country_map.items() }
# Example: name = 'Armenia', iso = 'ARM', code = '0260'

# add code 0575R to the COUNTRIES_CODE dictionary
country_map['ECARO'] = '575R'
country_code_map['ECARO, Switzerland'] = '575R'


mapping = get_ecaro_countries_mapping(keys="code", values="iso")

In [ ]:
dfs = []
# sheet name to read
sheet_name = 'Prorated Goal Area'
# filename and number of rows to skip


files = [
    (PROJ_ROOT / data_source[0], 5),
    (PROJ_ROOT / data_source[1], 4)
]

def read_and_process_excel(file_path, sheet_name, skiprows):
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprows)
    df.rename(lambda x: x.lower().replace(' ', '_'), axis=1, inplace=True)
    df = df[df['row_labels'] != 'Grand Total']
    df.dropna(subset=['row_labels'], inplace=True)
    return df

for file_name, skip in files:

    dfs.append(read_and_process_excel(file_name, sheet_name, skip))

df_goals = pd.concat(dfs, axis=0, ignore_index=True)

print('Shape:', df_goals.shape)

In [ ]:
def check_df_type(df):
    for column in df.columns:
        print(df[column].dtype)

Clean, rename columns and assign activities to SDG

In [ ]:
# clean, rename columns
df_goals.rename(columns={'other_resources_-_emergency': 'OR-Emergency', 'other_resources_-_regular':'OR', 
                         'regular_resources':'RR'}, inplace=True)

numeric_columns_list = ['OR-Emergency', 'OR', 'RR', 'grand_total']

# replace missing numeric entries with 0
df_goals[numeric_columns_list] = df_goals[numeric_columns_list].fillna(0.)

# There is an activity labelled 'Unknown' with a grand total of ~ 12 million USD in current CP (0 USD in previous CP), 
# likely as those funds were not yet assigned to activities as latest CP cycle is not over yet. 
# We will filter for all activities where we do not have information 

print('Shape before:', df_goals.shape)

df_goals[['activity', 'activity_wbs', 'type']] = None
# Make a column with the type of row_entries: goal and activity
df_goals['type'] = df_goals['row_labels'].apply(
    lambda x: 'goal' if x.startswith('Goal') 
    else 'activity'
)

# create activity rows with codes and names
df_goals.loc[df_goals['type'] == 'activity', 'activity_wbs'] = df_goals['row_labels'].str.split(
    r'\s+', n=1, regex=True).str.get(0)
df_goals.loc[df_goals['type'] == 'activity', 'activity'] = df_goals['row_labels'].str.split(
    r'\s+', n=1, regex=True).str.get(-1)

df_goals[['activity_wbs', 'activity']] = df_goals[['activity_wbs', 'activity']].ffill()
# remove rows with type == activity (only goals and unknown remain)
df_goals = df_goals[df_goals['type'] != 'activity']

# add 'goal_area_code' from row_labels, 'goal_area' via dictionary SDG_goals, and rename row_labels to goals
df_goals[['goal_area_code']] = df_goals['row_labels'].str.extract(r'Goal (\d+)')
df_goals['goal_area'] = df_goals['goal_area_code'].map(SDG_goals)
df_goals.rename(columns={'row_labels': 'goals'}, inplace=True)

# we should be left with roughly half the entries (some activities have several goals)
print('Shape after:', df_goals.shape)

Unpivot df_goals from wide to long format

In [ ]:
df_goals = pd.melt(df_goals, 
                    id_vars=['goals', 'goal_area','goal_area_code', 'activity', 'activity_wbs'], 
                    value_vars=['OR', 'OR-Emergency', 'RR'], 
                    var_name='funds_type', 
                    value_name='funds', 
                    ignore_index=True)

print('Unpivoted df:', df_goals.shape)

Filter out funds with values ~ 0 USD (funds < abs(0.01))

In [ ]:
df_goals['funds'] = df_goals['funds'].astype(float) # make sure column is type float
df_goals = df_goals[abs(df_goals['funds']) > 0.01]
print('Shape after dropping funds ~ 0:', df_goals.shape)

Add Countries and CP from dictionaries and activities_programme_stucture.xlsx:

In [ ]:
# classify the cycles based on a dictionary:
cp_dictionary = {'ARM04': 'CP (2010-2015)', 'ARM05': 'CP (2016-2021)', 'ARM06': 'CP (2021-2025)',
                 'AZE04': 'CP (2011-2016)', 'AZE05': 'CP (2016-2021)', 'AZE06': 'CP (2021-2025)',
                 'BIH06': 'CP (2010-2014)', 'BIH07': 'CP (2015-2020)', 'BIH08': 'CP (2021-2025)',
                 'BLR04': 'CP (2011-2015)', 'BLR05': 'CP (2016-2020)', 'BLR06': 'CP (2021-2025)',                 
                 'GEO04': 'CP (2011-2015)', 'GEO05': 'CP (2016-2021)', 'GEO06': 'CP (2021-2025)',
                 'KAZ04': 'CP (2010-2015)', 'KAZ05': 'CP (2016-2020)', 'KAZ06': 'CP (2021-2025)',
                 'MKD05': 'CP (2010-2015)', 'MKD06': 'CP (2016-2021)', 'MKD07': 'CP (2021-2025)'
                 }


In [ ]:
df_goals['business_area'] = df_goals['activity_wbs'].apply(
    lambda x: 'Unknown' if x == 'Unknown' 
    else x.split("/")[0])

In [ ]:
# add country name from dictionary
df_goals['country'] = df_goals['business_area'].apply(
    lambda x: 'Unknown' if x == 'Unknown' 
    else mapping.get(x))

In [ ]:

# remove countries not in the list
df_goals = df_goals[df_goals['country'].eq(COUNTRY)]
# add CP from dictionary
df_goals['cp'] = (df_goals['country'] + df_goals['activity_wbs'].str.split('/').str[2]).map(cp_dictionary)
# drop CP which are not in dictionary (before 2011 or Unknown)
df_goals.dropna(subset=['cp'], inplace=True)
df_goals['start_year'] = df_goals['cp'].str.extract(r'(\d{4})')
# remove early CP (before 2015/2016)
df_goals['start_year'] = df_goals['start_year'].astype(int)
print('Shape:', df_goals.shape)
mask = df_goals['start_year'] > 2012
df_goals = df_goals[mask]
print('Remove CP before 2012:', df_goals.shape)

Add 2016 - 2017 data without SDG information:

- if there is a 1 <-> 1 mapping of activity to SDG Goal, then we use that to map the funds in the 2016-2017 data
    
- if the mapping is not 1 <-> 1, we label the activity SDG as being 'Undefined' (Goal Are Code 0) and add the funds to the df

In [ ]:

file_path = PROJ_ROOT / data_source[2]

sheet_name = 'Prorated Outcome Area'
df_addfunds = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=5)
df_addfunds.rename(lambda x: x.lower().replace(' ', '_'), axis=1, inplace=True)
df_addfunds = df_addfunds[df_addfunds['row_labels'] != 'Grand Total']
df_addfunds.dropna(subset=['row_labels'], inplace=True)
print(df_addfunds.shape)

In [ ]:
# clean, rename columns
df_addfunds.rename(columns={'other_resources_-_emergency': 'OR-Emergency', 'other_resources_-_regular':'OR', 
                         'regular_resources':'RR'}, inplace=True)

numeric_columns_list = ['OR-Emergency', 'OR', 'RR', 'grand_total']

# replace missing numeric entries with 0
df_addfunds[numeric_columns_list] = df_addfunds[numeric_columns_list].fillna(0.)

In [ ]:
print('Shape before:', df_addfunds.shape)
# create activity rows with codes and names
df_addfunds['activity_wbs'] = df_addfunds['row_labels'].str.split(r'\s+', n=1, regex=True).str.get(0)
df_addfunds['activity'] = df_addfunds['row_labels'].str.split(r'\s+', n=1, regex=True).str.get(-1)

# add country name from dictionary
df_addfunds['country'] = df_addfunds['activity_wbs'].apply(
    lambda x: 'Unknown' if x == 'Unknown' 
    else next((code for code, value in country_map.items() if x.startswith(value + '/')), None))

# remove countries not in the list, will also remove 'Unknown'
df_addfunds = df_addfunds[df_addfunds['country'].eq(COUNTRY)]

print('Shape after:', df_addfunds.shape)

In [ ]:
# Unpivot df_addfunds from wide to long format
df_addfunds = pd.melt(df_addfunds, 
                    id_vars=['country', 'activity', 'activity_wbs'], 
                    value_vars=['OR', 'OR-Emergency', 'RR'], 
                    var_name='funds_type', 
                    value_name='funds', 
                    ignore_index=True)

print('Unpivoted df:', df_addfunds.shape)

In [ ]:
# Filter out funds with values ~ 0 USD (funds < abs(0.01))
df_addfunds['funds'] = df_addfunds['funds'].astype(float) # make sure column is type float
df_addfunds = df_addfunds[abs(df_addfunds['funds']) > 0.01]
print('Shape after dropping funds ~ 0:', df_addfunds.shape)

In [ ]:
# add CP from dictionary
df_addfunds['cp'] = (df_addfunds['country'] + df_addfunds['activity_wbs'].str.split('/').str[2]).map(cp_dictionary)
# drop CP which are not in dictionary (before 2011 or Unknown)
df_addfunds.dropna(subset=['cp'], inplace=True)

df_addfunds['start_year'] = df_addfunds['cp'].str.extract(r'(\d{4})')
# remove early CP (before 2015/2016)
df_addfunds['start_year'] = df_addfunds['start_year'].astype(int)
print('Shape:', df_addfunds.shape)
# Keep only last 2 CP (start year  > 2012)
mask = df_addfunds['start_year'] > 2012
df_addfunds = df_addfunds[mask]
print('Remove CP before 2012:', df_addfunds.shape)

Create a mapping of activity WBS <-> SDG goal, to use when we don't have SDG data

In [ ]:
df_mapping = df_goals[['activity_wbs','goal_area_code']].copy()
print('Before', df_mapping.shape)
# keep only unique WBS <-> SDG mappings
df_mapping.drop_duplicates(inplace=True)

# Concatenate values with a comma (or any separator you prefer)
df_mapping = df_mapping.groupby('activity_wbs', as_index=False).agg({'goal_area_code': lambda x: ', '.join(x)})
# if there are multiple sdg_goal_codes then replace with 'Undefined'
df_mapping['goal_area_code'] = df_mapping['goal_area_code'].apply(lambda x: 'Undefined' if ',' in str(x) else x)
print('After', df_mapping.shape)
df_mapping['goal_area'] = df_mapping['goal_area_code'].map(SDG_goals)
df_mapping['goal_area'] = df_mapping['goal_area'].fillna('Undefined')

In [ ]:
# Add the SDG mapping to the 2016-2017 funds, 'Undefined' is also used when we have no information
df_addfunds = df_addfunds.merge(df_mapping, on='activity_wbs', how='left')
df_addfunds.fillna({'goal_area': 'Undefined', 'goal_area_code': 'Undefined'}, inplace=True)
print(df_addfunds.shape)

In [ ]:
print('df_addfunds', df_addfunds.shape)
print('df_goals', df_goals.shape)
column_order = ['country', 'activity', 'activity_wbs', 'funds_type', 'funds', 'cp',
       'start_year', 'goal_area_code', 'goal_area']
df_goals = df_goals[column_order]
df_addfunds = df_addfunds[column_order]
# Combine the 2016-2017 (df_addfunds) data with inferred SDG and the 2018-2024 data (df_goals) where we know SDG
df_combined = pd.concat([df_goals, df_addfunds], ignore_index=True)
print(df_combined.shape)
df_combined.sort_values(by=['country','start_year' ], inplace=True, ascending=[True, True])

In [ ]:
# check for duplicated funds, if the shape is the same all funds are different and we didn't duplicate funds by adding
# the 2016-2017 data
print(df_combined.shape)
df_combined.drop_duplicates(subset=['activity_wbs', 'funds_type', 'funds'], inplace=True)
print(df_combined.shape)

**Write to Spreadsheet**

In [ ]:
df_combined.to_excel(product['data'], index=False)

::: {.callout-note}
Expenses are from the Startegic Plan for years 2016-2024 and do not correspond directly to Funds Utilization in the Country Programme Cycles.
:::